In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import pandas as pd
import os

In [2]:
#display setup for visibility
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [19]:
#connecting with chrome 
service = Service('./chromedriver.exe')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get('https://europeanleague.football/schedule?season=2022')
time.sleep(3)
#clicking through cookies
driver.find_element(By.XPATH,'.//*[@id="consentDialog"]/div[2]/div[2]/div/div[2]/div/div[1]/div').click()
time.sleep(3)
page_source = driver.page_source
time.sleep(5)

In [20]:
#connecting to BS4
soup = BeautifulSoup(page_source, 'lxml')

In [24]:
#creating data frames
#df with general info
games = [g.get_text() for g in soup.select('h4')]
games_sublists = [games[i:i+10] for i in range(0, len(games), 10)]
df_games = pd.DataFrame(games_sublists, columns=['season','month','date_full','game_hour','time_zone',
                                                 'stadium_city','away_team','drop1','home_team','drop2'])
#df with scores
scores = soup.find_all('div', {'class':'g-score is-real'})
score = [s.get_text() for s in scores]
score = [scores[i:i+2] for i in range(0, len(scores), 2)]
df_scores = pd.DataFrame(scores_sublist, columns=['home_team_final_score','away_team_final_score'])
#df for weeks
weeks = soup.find_all('div', {'class':'gd-no'})
week = [w.get_text() for w in weeks]
week_sublist = [week[i+1:i+2] for i in range(0, len(week), 2)]
df_weeks = pd.DataFrame(week_sublist, columns=['game_week'])
#df merging above frames
df_game_info = pd.concat([df_games, df_scores, df_weeks], axis=1)
df_game_info = df_game_info[df_game_info['season'].str.contains('20')]
df_game_info.drop(['drop1', 'drop2'], axis=1,inplace=True)
df_game_info

,season,month,date_full,game_hour,time_zone,stadium_city,away_team,home_team,home_team_final_score,away_team_final_score,game_week
0,2021,June,"Saturday, June 19, 2021",18:00,CET,,Cologne Centurions,Panthers Wroclaw,39,55,1
1,2021,June,"Saturday, June 19, 2021",19:00,CET,,Stuttgart Surge,Barcelona Dragons,21,17,1
2,2021,June,"Sunday, June 20, 2021",15:00,CET,,Frankfurt Galaxy,Hamburg Sea Devils,15,17,1
3,2021,June,"Sunday, June 20, 2021",15:00,CET,,Leipzig Kings,Berlin Thunder,37,27,1
4,2021,June,"Saturday, June 26, 2021",18:00,CET,,Leipzig Kings,Panthers Wroclaw,28,54,1
5,2021,June,"Sunday, June 27, 2021",15:00,CET,,Barcelona Dragons,Cologne Centurions,12,40,1
6,2021,June,"Sunday, June 27, 2021",15:00,CET,,Frankfurt Galaxy,Stuttgart Surge,42,20,2
7,2021,July,"Saturday, July 3, 2021",19:00,CET,,Barcelona Dragons,Hamburg Sea Devils,32,14,3
8,2021,July,"Sunday, July 4, 2021",15:00,CET,,Panthers Wroclaw,Frankfurt Galaxy,13,22,3
9,2021,July,"Sunday, July 4, 2021",15:00,CET,,Cologne Centurions,Leipzig Kings,48,47,3


In [26]:
#saving data to csv file
if not os.path.isfile('elf_schedule_scrap.csv'):
    df_game_info.to_csv('elf_schedule_scrap.csv', header='column_names')
else: # else it exists so append without writing the header
    df_game_info.to_csv('elf_schedule_scrap.csv', mode='a', header=False)